In [ ]:
from tkinter import *
from tkinter import filedialog, messagebox
import pygame
import os
import math

# CONSTANTS
PINK = "#E2979C"
RED = "#E7305B"
ORANGE = "#F77F00"
YELLOW = "#FFE893"
GREEN = "#87A922"
FONT_NAME = "Courier"
WORK_MIN = 1
SHORT_BREAK_MIN = 1
LONG_BREAK_MIN = 20
REPS = 0
TIMER = NONE


# TIMER RESET
def reset_timer():
    global TIMER,REPS
    root.after_cancel(TIMER)
    canvas.itemconfig(timer_text, text="00:00")
    title_label.config(text="Timer")
    check_mark.config(text="")
    
# TIMER MECHANISM
def start_timer():
    global REPS
    REPS += 1
    
    work_sec = WORK_MIN*60
    short_work_sec= SHORT_BREAK_MIN*60
    long_break_sec = LONG_BREAK_MIN*60

    if REPS%8==0:
        count_down(long_break_sec)
        title_label.config(text="Break", fg=RED)
    elif REPS%2 == 0:
        count_down(short_work_sec)
        title_label.config(text="Break", fg=PINK)
    else:
        count_down(work_sec)
        title_label.config(text="Work", fg=GREEN)

    
    
# COUNTDOWN MECHANISM
def count_down(count):
    global REPS, TIMER
    count_min = math.floor(count/60)
    count_sec = count%60
    if count_sec<10:
        count_sec = f"0{count_sec}"
    canvas.itemconfig(timer_text, text=f"{count_min}:{count_sec}")
    if count>=0:
        global TIMER
        TIMER = root.after(1000, count_down, count-1)
    else:
        # Bring the window to front when timer hits 00:00
        root.deiconify()               # Restore window if minimized
        root.lift()                    # Raise to top
        root.attributes('-topmost', 1) # Force on top
        start_timer()
        marks = ""
        work_sessions = math.floor(REPS/2)
        for i in range(work_sessions):
            marks += "✔"
        if len(marks)>4:
            marks = "✔"
        check_mark.config(text = marks)
            

# UI SETUP
root = Tk()
root.title("Pomodoro")
root.geometry("380x600")  # Increased height further
root.config(padx=40,pady=50, bg=YELLOW)
root.resizable(False, False) # Disable resizing in both width and height


# Configure grid to expand properly
root.grid_rowconfigure(4, weight=1)  # Make row 4 expandable
root.grid_columnconfigure(1, weight=1)

# Main content frame to hold timer components
main_frame = Frame(root, bg=YELLOW)
main_frame.grid(row=0, column=0, columnspan=3, sticky="nsew")

title_label = Label(main_frame, text="Timer", fg=GREEN, bg=YELLOW)
title_label.configure(font=(FONT_NAME, 36, "bold"))
title_label.grid(row=0, column=1)

canvas = Canvas(main_frame, width=200, height=224, bg=YELLOW, highlightthickness=0)
tomato_img = PhotoImage(file="tomato.png")
# x,y coordinates of the center of image w.r.t canvas
canvas.create_image(100, 112, image=tomato_img)
# *args(x,y coordinates of center of text) ,**kwargs(texts)
timer_text = canvas.create_text(100, 130, text="00:00", fill="white", font=(FONT_NAME, 35, "bold"))
canvas.grid(row=1, column=1)

start_btn = Button(main_frame, text="Start", highlightthickness=0,
                  bg=GREEN, fg="white", font=(FONT_NAME, 10), command=start_timer)
start_btn.grid(row=2, column=0)

reset_btn = Button(main_frame, text="Reset", highlightthickness=0,
                  bg=GREEN, fg="white", font=(FONT_NAME, 10), command = reset_timer)
reset_btn.grid(row=2, column=2)

check_mark = Label(main_frame, fg=GREEN, bg=YELLOW, font=(FONT_NAME, 14, "bold"))
check_mark.grid(row=3, column=1)

sessions_done = Label(main_frame, text="\nSessions \nCompleted: ", fg=GREEN, bg=YELLOW, font=(FONT_NAME, 14))
sessions_done.grid(row=2, column=1)

# Create a separator frame
separator = Frame(root, height=2, bg=GREEN)
separator.grid(row=4, column=0, columnspan=3, sticky="ew", pady=20)

# LOFI MUSIC - Bottom frame to ensure controls are at the bottom
bottom_frame = Frame(root, bg=YELLOW)
bottom_frame.grid(row=5, column=0, columnspan=3, sticky="ew")

# Make sure pygame is initialized properly
try:
    pygame.mixer.init()
    print("Pygame mixer initialized successfully")
except Exception as e:
    print(f"Failed to initialize pygame mixer: {str(e)}")
    messagebox.showerror("Error", f"Failed to initialize audio system: {str(e)}")

# Use the current directory as the music folder
root.directory = f"{os.getcwd()}\\music"  # Gets the current working directory

# Music player controls
# Load original images and scale them down appropriately
play_btn_img_original = PhotoImage(file="play.png")
pause_btn_img_original = PhotoImage(file="pause.png")
next_btn_img_original = PhotoImage(file="next.png")
prev_btn_img_original = PhotoImage(file="previous.png")

# Calculate subsample rates based on original size
# For extremely large images: 1280x1280 -> subsample by 32 to get roughly 40x40
play_btn_img = play_btn_img_original.subsample(32, 32)
pause_btn_img = pause_btn_img_original.subsample(32, 32)
next_btn_img = next_btn_img_original.subsample(25, 25)  
prev_btn_img = prev_btn_img_original.subsample(25, 25)

# Control frame specifically for music buttons
control_frame = Frame(bottom_frame, bg=YELLOW)
control_frame.pack(pady=10)

# Create buttons with properly scaled images - re-arranged order to prev, play/pause, next
prev_btn = Button(control_frame, image=prev_btn_img, borderwidth=0, relief="solid", highlightthickness=0)
play_btn = Button(control_frame, image=play_btn_img, borderwidth=0, relief="solid", highlightthickness=0)
pause_btn = Button(control_frame, image=pause_btn_img, borderwidth=0, relief="solid", highlightthickness=0)
next_btn = Button(control_frame, image=next_btn_img, borderwidth=0, relief="solid", highlightthickness=0)

# Only show play button initially, hide pause button
play_btn.grid(row=0, column=1, padx=7, pady=10)
prev_btn.grid(row=0, column=0, padx=7, pady=10)
next_btn.grid(row=0, column=2, padx=7, pady=10)
# Don't grid the pause button initially

# Current song label
current_song_label = Label(bottom_frame, text="No Music Folder Selected", bg=YELLOW, fg=GREEN)
current_song_label.pack(pady=5)

songs = []
current_song = ""
paused = False
i = 0  # Track the current song index

def load_music():
    """Load music files from the selected directory"""
    global songs, current_song
    if not root.directory:
        current_song_label.config(text="No music folder selected")
        return
        
    songs = []  # Clear previous songs list
    for song in os.listdir(root.directory):
        name, ext = os.path.splitext(song)  # Fixed splittext to splitext
        if ext == ".mp3":
            songs.append(song)
    
    if songs:
        current_song = songs[0]
        current_song_label.config(text=f"Ready to play: {current_song}")
    else:
        current_song_label.config(text="No MP3 files found in selected folder")

# Toggle state variable
is_playing = False

# when song ends autoplay to next
def check_music_end():
    if is_playing and not pygame.mixer.music.get_busy():
        next_music()  # Move to the next song
    root.after(1000, check_music_end)  # Check again after 1 second


# Function to toggle between play and pause
def toggle_play_pause():
    global is_playing
    if is_playing:
        # Currently playing, so pause it
        is_playing = False
        pause_btn.grid_forget()  # Hide pause button
        play_btn.grid(row=0, column=1, padx=7, pady=10)  # Show play button
        pause_music()
        current_song_label.config(text=f"Paused: {current_song}")
    else:
        # Currently paused, so play it
        is_playing = True
        play_btn.grid_forget()  # Hide play button
        pause_btn.grid(row=0, column=1, padx=7, pady=10)  # Show pause button
        play_music()
        current_song_label.config(text=f"Now Playing: {current_song}")

def play_music():
    global current_song, paused
    if not songs:
        current_song_label.config(text="No songs available to play")
        return
        
    if not paused:
        pygame.mixer.music.load(os.path.join(root.directory, current_song))
        pygame.mixer.music.play()
    else:
        pygame.mixer.music.unpause()
        paused = False
    check_music_end()  # Start checking for song end
        
def pause_music():
    global paused
    pygame.mixer.music.pause()
    paused = True
    
# Functions for music controls
def next_music():
    global i, current_song, paused
    if not songs:
        return
 
    i += 1
    if i >= len(songs):
        i = 0  # Loop back to the first song
    current_song = songs[i]
    paused = False  # Reset paused state
    play_music()
    current_song_label.config(text=f"Now Playing: {current_song}")

        
def prev_music():
    global i, current_song, paused
    if not songs:
        return
 
    i -= 1
    if i < 0:
        i = len(songs) - 1  # Loop to the last song
    current_song = songs[i]
    paused = False  # Reset paused state
    play_music()
    current_song_label.config(text=f"Now Playing: {current_song}")
 

# Assign commands to buttons
play_btn.config(command=toggle_play_pause)
pause_btn.config(command=toggle_play_pause)
next_btn.config(command=next_music)
prev_btn.config(command=prev_music)

# Load music on startup
load_music()

# Now update the label with the result of loading
if songs:
    current_song_label.config(text=f"Ready to play: {current_song}")
else:
    current_song_label.config(text="No MP3 files found in current directory")


play_music()
toggle_play_pause()
# Function to handle window closing
def on_closing():
    # Stop any playing music
    if pygame.mixer.music.get_busy():
        pygame.mixer.music.stop()
    # Quit pygame properly
    pygame.mixer.quit()
    pygame.quit()
    root.quit()
    root.destroy()
    os._exit(0) 

# Set the closing protocol
root.protocol("WM_DELETE_WINDOW", on_closing)

# Start the main loop
root.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.13.1)
Hello from the pygame community. https://www.pygame.org/contribute.html
Pygame mixer initialized successfully
